In [1]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import json
from copy import copy

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Autosaving every 300 seconds


# Data Cleaning and Preliminary Exploration

## Preliminary Exploration

In [2]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,...,NaN,NaN,1.36926,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.498289,5.96721,6,6,2017-08-13 23:50:48.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.999316,11.96720,12,12,2017-08-13 23:50:48.0
4,3,011_S_0003,m24,11,ADNI1,ADNI1,2007-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,1.998630,23.93440,24,24,2017-08-13 23:50:48.0


#### NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
#### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

In [3]:
''' ADNI2 data within ADNIMERGE.csv '''
# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)
ADNI2.head()


(6937, 94)


,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,Male,...,1.00000,1.25641,1.13549,NaN,1.326790,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,Male,...,1.33333,1.37838,1.50629,NaN,0.985156,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


In [4]:
# It seems there were some 'unknown'..I replaced them with nan
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)
ADNI2 = ADNI2.replace(-1, np.nan)
ADNI2 = ADNI2.replace(-4, np.nan)

# From now on just bl


In [5]:
ADNI2_bl = ADNI2.copy()
ADNI2_bl = ADNI2_bl[ADNI2_bl['VISCODE'] == 'bl']

In [6]:
ADNI2_bl.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'FDG', 'PIB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
       'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl

In [7]:
ADNI2_bl.shape

(789, 94)

In [8]:
#Drop the ones w/o bl
#I kept 'RID', 'PTID', 'VISCODE', 'COLPROT','DX_bl'
#'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
#'PTMARRY', 'APOE4', 'FDG', , ADAS11_bl',
# 'ADAS13_bl', 'CDRSB_bl', 'MMSE_bl',
#'RAVLT_immediate_bl', 'RAVLT_learning_bl',
#'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
#'FLDSTRENG_bl', , 'Ventricles_bl', 'Hippocampus_bl',
#'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
#'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
#'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
#'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
#'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
#'PIB_bl', 'AV45_bl'


ADNI2_bl= ADNI2_bl.drop(labels = [ 'SITE', 'ORIGPROT', 'EXAMDATE',
         'AV45', 'CDRSB', 'ADAS11', 'ADAS13','PIB',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
       'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl',   'Years_bl', 'Month_bl', 'Month', 'M',
       'update_stamp', 'FSVERSION_bl', 'FLDSTRENG_bl'],axis=1)

# Drop beacuse same values 'FSVERSION_bl', 'FLDSTRENG_bl'

In [9]:
ADNI2_bl.columns

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTGENDER',
       'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'PIB_bl', 'AV45_bl'],
      dtype='object')

In [10]:
# calculate the % of each column that is NaNs and print in descending order
print((ADNI2_bl.isnull().sum()/len(ADNI2_bl['DX_bl'])).sort_values(ascending=False))

PIB_bl                      1.000000
Entorhinal_bl               0.136882
Fusiform_bl                 0.136882
MidTemp_bl                  0.136882
Hippocampus_bl              0.112801
Ventricles_bl               0.081115
EcogSPOrgan_bl              0.050697
WholeBrain_bl               0.046895
EcogSPDivatt_bl             0.032953
EcogPtOrgan_bl              0.030418
AV45_bl                     0.029151
EcogSPVisspat_bl            0.026616
EcogSPPlan_bl               0.022814
FDG                         0.019011
FDG_bl                      0.019011
ICV_bl                      0.017744
RAVLT_forgetting_bl         0.016477
RAVLT_learning_bl           0.013942
APOE4                       0.013942
EcogPtDivatt_bl             0.013942
EcogSPTotal_bl              0.013942
EcogSPMem_bl                0.012674
MOCA_bl                     0.012674
EcogSPLang_bl               0.011407
EcogPtVisspat_bl            0.010139
FAQ_bl                      0.007605
ADAS13_bl                   0.007605
E

In [11]:
#Drop 50% or more missing
ADNI2_bl= ADNI2_bl.dropna(axis=1, how='any', thresh=len(ADNI2_bl['DX_bl'])/2, inplace=False)

In [12]:
encode = preprocessing.LabelEncoder()
ADNI2_bl.loc[:, 'DX_bl'] = encode.fit_transform(ADNI2_bl['DX_bl'])

In [13]:
ADNI2_bl.head()

,RID,PTID,VISCODE,COLPROT,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,...,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,AV45_bl
58,5282,082_S_5282,bl,ADNI2,4,66.9,Male,17,Not Hisp/Latino,White,...,NaN,1.625,1.33333,1.14286,1.2,1.00000,1.00000,1.25641,1.13549,1.326790
91,5280,100_S_5280,bl,ADNI2,4,67.5,Male,16,Not Hisp/Latino,Black,...,1.43590,1.625,1.11111,1.28571,1.0,1.25000,1.00000,1.25000,1.25195,0.983143
92,5279,082_S_5279,bl,ADNI2,4,68.5,Male,20,Not Hisp/Latino,White,...,1.25641,1.875,1.22222,1.28571,1.0,1.33333,1.33333,1.37838,1.50629,0.985156
142,5278,082_S_5278,bl,ADNI2,4,80.2,Male,19,Not Hisp/Latino,White,...,1.30769,1.250,1.00000,1.00000,1.0,1.00000,1.00000,1.05128,1.22045,1.226230
174,5277,027_S_5277,bl,ADNI2,4,71.8,Female,15,Not Hisp/Latino,White,...,1.84211,1.875,1.00000,1.00000,1.0,1.00000,1.75000,1.25641,1.42276,1.734130


In [14]:
#hot-one encoding for categoricla and clean

categorical_predictors = ['PTETHCAT', 'PTRACCAT', 'PTMARRY','APOE4', 'PTGENDER']



''' Remove all categorical data with nan (only ~0.5%) '''
missing_rows = np.nonzero(np.sum(ADNI2_bl[categorical_predictors].isnull(), axis=1))
ADNI2_bl = ADNI2_bl.drop(ADNI2_bl.index[missing_rows[0]])

'''' Hot-One Encoding of Categorical Predictors '''
ADNI2_bl_categorical = pd.get_dummies(ADNI2_bl[categorical_predictors] , columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
ADNI2_bl= pd.get_dummies(ADNI2_bl, columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)



### Removing All Missing Data

In [15]:
ADNI2_bl_remove_all_missing = ADNI2_bl.copy()
for column in ADNI2_bl.columns:
    if ADNI2_bl[column].isnull().any():
        ADNI2_remove_all_missing = ADNI2_bl.drop(labels=column, axis=1)

In [16]:
ADNI2_bl_remove_all_missing.head()

,RID,PTID,VISCODE,COLPROT,DX_bl,AGE,PTEDUCAT,FDG,CDRSB_bl,ADAS11_bl,...,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,PTGENDER_Female,PTGENDER_Male
58,5282,082_S_5282,bl,ADNI2,4,66.9,17,1.13549,0.0,10.0,...,1,0,1,0,0,0,1,0,0,1
91,5280,100_S_5280,bl,ADNI2,4,67.5,16,1.25195,0.0,3.0,...,0,0,0,1,0,1,0,0,0,1
92,5279,082_S_5279,bl,ADNI2,4,68.5,20,1.50629,0.0,4.0,...,1,0,1,0,0,1,0,0,0,1
142,5278,082_S_5278,bl,ADNI2,4,80.2,19,1.22045,0.0,3.0,...,1,0,0,1,0,0,1,0,0,1
174,5277,027_S_5277,bl,ADNI2,4,71.8,15,1.42276,0.5,4.0,...,1,0,0,0,1,0,1,0,1,0


In [17]:
ADNI2_bl_remove_all_missing.shape

(771, 58)

In [18]:
ADNI2_bl_remove_all_missing.columns

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0',

In [19]:
print(ADNI2_bl_remove_all_missing.shape)
ADNI2_bl_remove_all_missing.head()


(771, 58)


,RID,PTID,VISCODE,COLPROT,DX_bl,AGE,PTEDUCAT,FDG,CDRSB_bl,ADAS11_bl,...,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,PTGENDER_Female,PTGENDER_Male
58,5282,082_S_5282,bl,ADNI2,4,66.9,17,1.13549,0.0,10.0,...,1,0,1,0,0,0,1,0,0,1
91,5280,100_S_5280,bl,ADNI2,4,67.5,16,1.25195,0.0,3.0,...,0,0,0,1,0,1,0,0,0,1
92,5279,082_S_5279,bl,ADNI2,4,68.5,20,1.50629,0.0,4.0,...,1,0,1,0,0,1,0,0,0,1
142,5278,082_S_5278,bl,ADNI2,4,80.2,19,1.22045,0.0,3.0,...,1,0,0,1,0,0,1,0,0,1
174,5277,027_S_5277,bl,ADNI2,4,71.8,15,1.42276,0.5,4.0,...,1,0,0,0,1,0,1,0,1,0


In [20]:
ADNI2_bl_remove_all_missing.columns

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0',

In [21]:
ADNI2_bl_remove_all_missing.columns.isnull()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False], dtype=bool)

In [22]:
ADNI2_bl_remove_all_missing.columns

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0',

In [23]:
ADNI2_bl_remove_all_missing.to_csv(path_or_buf='ADNI2_bl_remove_all_missing.csv', index=False)

## Model-Based Imputation 
- Drop around 5% of Categorical predictors and use Linear Regression for missing continuous predictors

In [24]:
ADNI2_bl_impute = ADNI2_bl.copy()
ADNI2_bl_impute.columns

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0',

In [25]:
all_predictors = ['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTGENDER',
       'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0']

#For the continuos I had to remove
#'RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 
#'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
#'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
#'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
#'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
#'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
#'APOE4_2.0'
    
continuous_predictors = ['AGE', 
       'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       ]

In [26]:
''' Create ADNI2_missing for imputation '''
ADNI2_bl_impute_remove_categorical = ADNI2_bl_impute[continuous_predictors]

''' Create the Dataframe for imputation '''
ADNI2_bl_missing = pd.concat([ADNI2_bl['VISCODE'], ADNI2_bl_impute_remove_categorical, ADNI2_bl_categorical], axis=1)


In [27]:
print(ADNI2_bl_impute_remove_categorical.shape)
print(ADNI2_bl_categorical.shape)

(771, 35)
(771, 17)


In [28]:
ADNI2_bl_missing.head()

,VISCODE,AGE,PTEDUCAT,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,...,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,PTGENDER_Female,PTGENDER_Male
58,bl,66.9,17,0.0,10.0,15.0,29,42.0,5.0,2.0,...,1,0,1,0,0,0,1,0,0,1
91,bl,67.5,16,0.0,3.0,5.0,30,42.0,7.0,2.0,...,0,0,0,1,0,1,0,0,0,1
92,bl,68.5,20,0.0,4.0,7.0,30,61.0,6.0,3.0,...,1,0,1,0,0,1,0,0,0,1
142,bl,80.2,19,0.0,3.0,10.0,29,36.0,8.0,7.0,...,1,0,0,1,0,0,1,0,0,1
174,bl,71.8,15,0.5,4.0,6.0,28,44.0,8.0,4.0,...,1,0,0,0,1,0,1,0,1,0


In [29]:
#hsiang had this one
#'Years_bl',  

#I have these more
#  'Ventricles_bl', 'Hippocampus_bl',, 


In [30]:
print(ADNI2_bl_impute_remove_categorical.shape)

(771, 35)


In [31]:
print(ADNI2_bl_missing.columns)

Index(['VISCODE', 'AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl',
       'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0', 'PTGENDE

In [32]:
full_dataframe =  copy(ADNI2_bl_missing)
missing_rows = []
for i in range(ADNI2_bl_missing.shape[0]):
    if pd.isnull(ADNI2_bl_missing.values[i]).any():
        missing_rows.append(i)
full_dataframe = full_dataframe.drop(full_dataframe.index[missing_rows], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_bl_missing)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe.shape))

The origin number of data: 771
The number of data that contains missing entries: 274
The dimension of full dataframe: (497, 53)


In [33]:
''' To check that full_dataframe has no missing values '''
np.sum(np.sum(full_dataframe.isnull()))

0

In [34]:
ADNI2_bl_missing = ADNI2_bl_missing.drop(['VISCODE'], axis=1)
full_dataframe = full_dataframe.drop(['VISCODE'], axis=1)


In [35]:
full_dataframe.columns

Index(['AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
       'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
       'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
       'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
       'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
       'AV45_bl', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0', 'PTGENDER_Female', 'PTGEND

In [36]:
''' Model-Based Imputation - Linear Regression '''


lr = LinearRegression(fit_intercept = True)


for i in range(ADNI2_bl_missing.shape[0]):
    if ADNI2_bl_missing.iloc[i, :].isnull().any():
        missing_columns = ADNI2_bl_missing.columns[ADNI2_bl_missing.iloc[i, :].isnull()]
        #print(i, missing_columns)
        y_imp = full_dataframe[missing_columns]
        X_imp = full_dataframe.drop(missing_columns, 1)
        print(y_imp.shape)
        print(X_imp.shape)
        lr.fit(X_imp,y_imp)
        X_missing = ADNI2_bl_missing.iloc[i, :].drop(missing_columns)
        y_hat = lr.predict(X_missing)
        ADNI2_bl_missing.loc[ADNI2_bl_missing.index[i], missing_columns] = y_hat[0] 
     

(497, 13)
(497, 39)
(497, 3)
(497, 49)
(497, 3)
(497, 49)
(497, 9)
(497, 43)
(497, 2)
(497, 50)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 3)
(497, 49)
(497, 1)
(497, 51)
(497, 5)
(497, 47)
(497, 3)
(497, 49)
(497, 5)
(497, 47)
(497, 1)
(497, 51)
(497, 3)
(497, 49)
(497, 3)
(497, 49)
(497, 5)
(497, 47)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 4)
(497, 48)
(497, 4)
(497, 48)
(497, 4)
(497, 48)
(497, 6)
(497, 46)
(497, 5)
(497, 47)
(497, 5)
(497, 47)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 2)
(497, 50)
(497, 3)
(497, 49)
(497, 2)
(497, 50)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 5)
(497, 47)
(497, 3)
(497, 49)
(497, 3)
(497, 49)
(497, 3)
(497, 49)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 1)
(497, 51)
(497, 7)
(497, 45)
(497, 1)
(497, 51)
(497, 2)
(497, 50)
(497, 1)
(497, 51)
(497, 3)
(497, 49)
(497, 4)
(497, 48)
(497, 3)
(497, 49)
(497, 7)
(4

In [37]:
print(ADNI2_bl_missing.columns)

Index(['AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
       'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
       'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
       'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
       'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
       'AV45_bl', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0', 'PTGENDER_Female', 'PTGEND

In [38]:
ADNI2_bl_missing.shape

(771, 52)

In [39]:
ADNI2_bl_missing.head()

,AGE,PTEDUCAT,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,...,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,PTGENDER_Female,PTGENDER_Male
58,66.9,17,0.0,10.0,15.0,29,42.0,5.0,2.0,20.0000,...,1,0,1,0,0,0,1,0,0,1
91,67.5,16,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0000,...,0,0,0,1,0,1,0,0,0,1
92,68.5,20,0.0,4.0,7.0,30,61.0,6.0,3.0,20.0000,...,1,0,1,0,0,1,0,0,0,1
142,80.2,19,0.0,3.0,10.0,29,36.0,8.0,7.0,63.6364,...,1,0,0,1,0,0,1,0,0,1
174,71.8,15,0.5,4.0,6.0,28,44.0,8.0,4.0,40.0000,...,1,0,0,0,1,0,1,0,1,0


In [44]:
#pred_back =  ['PTID','RID', 'VISCODE', 'COLPROT', 'DX_bl']
pred_back =  [ 'DX_bl']
ADNI2_bl_missing [pred_back] = ADNI2_bl_remove_all_missing[pred_back]
print(np.sum(ADNI2_bl_missing.isnull()))

AGE                           0
PTEDUCAT                      0
CDRSB_bl                      0
ADAS11_bl                     0
ADAS13_bl                     0
MMSE_bl                       0
RAVLT_immediate_bl            0
RAVLT_learning_bl             0
RAVLT_forgetting_bl           0
RAVLT_perc_forgetting_bl      0
FAQ_bl                        0
Ventricles_bl                 0
Hippocampus_bl                0
WholeBrain_bl                 0
Entorhinal_bl                 0
Fusiform_bl                   0
MidTemp_bl                    0
ICV_bl                        0
MOCA_bl                       0
EcogPtMem_bl                  0
EcogPtLang_bl                 0
EcogPtVisspat_bl              0
EcogPtPlan_bl                 0
EcogPtOrgan_bl                0
EcogPtDivatt_bl               0
EcogPtTotal_bl                0
EcogSPMem_bl                  0
EcogSPLang_bl                 0
EcogSPVisspat_bl              0
EcogSPPlan_bl                 0
EcogSPOrgan_bl                0
EcogSPDi

In [43]:

ADNI2_bl_missing.to_csv(path_or_buf='ADNI2_bl_impute.csv', index=False)

# Below are More Preliminary Exploration by Carlo

                                                 **STRATEGY**

'RID',                    They should not be used for prediction, I'd keep them just for merging the prediction?
'PTID',                   They should not be used for prediction, I'd keep them just for merging the prediction?      
'VISCODE'                 They should not be used for prediction, I'd keep them just for merging the prediction?        'SITE'                    Shall we discard for the prediction?
'COLPROT'                 Shall we discard for the prediction?
'ORIGPROT'                Shall we discard for the prediction?
'EXAMDATE'                Shall we discard for the prediction?
'DX_bl'                   Hotone, categorical
'AGE',                    OK, int
'PTGENDER'                Hotone, categorical
'PTEDUCAT'                OK, int
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'                 Hotone, categorical
'APOE4'                   Hotone, categorical   Class of protein
'CDRSB'                   OK,float              Clinical Dementia Rating Scale–Sum of Boxes
'ADAS11'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'ADAS13'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'MMSE',                   OK,float              Mini Mental State Examination
'RAVLT_immediate'         OK,float              Rey Auditory Verbal Learning Test
'RAVLT_learning'          OK,float              Rey Auditory Verbal Learning Test
'RAVLT_forgetting',       OK,float              Rey Auditory Verbal Learning Test
'RAVLT_perc_forgetting'   OK,float              Rey Auditory Verbal Learning Test
'FAQ'                     OK,float              Functional Assessment Questionnaire
'MOCA'                    OK,float              Montreal Cognitive Assessment (MoCA)
'EcogPtMem'               OK,float              Cognitive
'EcogPtLang',             OK,float              Cognitive
'EcogPtVisspat'           OK,float              Cognitive
'EcogPtPlan'              OK,float              Cognitive
'EcogPtOrgan'             OK,float              Cognitive
'EcogPtDivatt'            OK,float              Cognitive
'EcogPtTotal'             OK,float              Cognitive
'EcogSPMem'               OK,float              Cognitive
'EcogSPLang'              OK,float              Cognitive
'EcogSPVisspat'           OK,float              Cognitive
'EcogSPPlan',             OK,float              Cognitive
'EcogSPOrgan'             OK,float              Cognitive
'EcogSPDivatt'            OK,float              Cognitive
'EcogSPTotal'             OK,float              Cognitive
'FSVERSION',              Just one value, discard
'Hippocampus'             OK,float              Hippocampus volume
'WholeBrain'              OK,float              Brain volume
'ICV'                     OK,float              Intracranial volume
'DX'                      OUR Y, ENCODING
'EXAMDATE_bl'             Shall we discard for the prediction?
'CDRSB_bl'                       ***FROM HERE BASELINE***
'ADAS11_bl'
'ADAS13_bl'
'MMSE_bl'
'RAVLT_immediate_bl'
'RAVLT_learning_bl'
'RAVLT_forgetting_bl'
'RAVLT_perc_forgetting_bl',
'FAQ_bl', 'FLDSTRENG_bl'
'FSVERSION_bl'
'Ventricles_bl',
'Hippocampus_bl'
'WholeBrain_bl'
'Entorhinal_bl'
'Fusiform_bl',
'MidTemp_bl'
'ICV_bl'
'MOCA_bl'
'EcogPtMem_bl'
'EcogPtLang_bl',
'EcogPtVisspat_bl'
'EcogPtPlan_bl'
'EcogPtOrgan_bl',
'EcogPtDivatt_bl'
'EcogPtTotal_bl'
'EcogSPMem_bl'
'EcogSPLang_bl',
'EcogSPVisspat_bl'
'EcogSPPlan_bl'
'EcogSPOrgan_bl',
'EcogSPDivatt_bl'
'EcogSPTotal_bl'
'FDG_bl'                   OK,float               fluorescence connect to PET
'AV45_bl'                  OK,float               average survival
'Years_bl',                OK,float               years form baseline
'Month_bl'                 Shall we discard for the prediction? We already have 'Years_bl'
'Month'                    Shall we discard for the prediction? We already have 'Years_bl'
'M'                        Shall we discard for the prediction? We already have 'Years_bl'
'update_stamp              Shall we discard for the prediction? 